# NLP - Checkpoint 02

1\. Contrua uma rede de relacionamento entre os personagens do livro: "Memórias Póstumas de Brás Cubas (1881)". Contando quantas vezes os nomes dos personagens aparecem juntos a cada conjunto de 20 palavras.

2\. Faça uma visualização da rede construída

3\. De acordo com sua opinião, quem são os personagens principais desse livro? 1º, 2º e 3º

Pergunta bônus: valendo 2 pontos no global solution: qual o personagem do texto que tem o estilo de texto mais próximo do bras cubas? E qual personagem tem o estilo de texto mais diferente?

Dica: revise os materiais das aulas 3 e 4, inclusive o código de comunidades em textos de rap.

## 1. Construção da rede de relacionamentos

In [66]:
import pandas
pandas.read_csv('relationships.csv').drop(columns=['Id'])

,Source,Target,Weight
0,Virgília,D. Plácida,10
1,Virgília,Lobo Neves,9
2,Sabina,Cotrim,8
3,Virgília,Quincas Borba,5
4,Virgília,Sabina,3
5,Sabina,Nhã-loló,2
6,Virgília,Marcela,2
7,Brás Cubas,Virgília,2
8,Lobo Neves,Quincas Borba,1
9,Lobo Neves,D. Plácida,1


## 2. Visualização da rede construída

<img src="relationships_graph.png">

## 3. Personagens principais

In [67]:
import pandas
pandas.read_csv('betweenness.csv').drop(columns=['Id'])

,Character,Betweenness
0,Virgília,0.463636
1,Brás Cubas,0.172727
2,Sabina,0.054545
3,Quincas Borba,0.036364
4,Marcela,0.018182
5,Lobo Neves,0.018182
6,Conselheiro Dutra,0.000000
7,Eugênia,0.000000
8,Nhã-loló,0.000000
9,D. Plácida,0.000000


## 4. Personagens com estilo de texto mais próximo e mais diferente

In [69]:
import pandas
pandas.read_csv('similarities.csv').drop(columns=['Id'])

,Character,Similarity
0,Lobo Neves,0.980151
1,Quincas Borba,0.975325
2,Marcela,0.973849
3,Virgília,0.973290
4,D. Plácida,0.970998
5,Cotrim,0.970131
6,Nhã-loló,0.965064
7,Eugênia,0.956754
8,Sabina,0.954632
9,Prudêncio,0.930684


# Imports and configurations

In [28]:
!pip3 install nltk

!pip3 install unidecode

!pip3 install pyvis pyvis-network

!pip3 install spacy
# !python3 -m spacy download pt_core_news_md
# !python3 -m spacy download pt_core_news_lg


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [29]:
from itertools import combinations
from typing import List

import networkx as nx
import nltk
import pandas as pd
import spacy
from nltk.corpus import machado
from nltk.text import Text
from pyvis.network import Network
from spacy.language import Language
from spacy.matcher import Matcher
from spacy.tokens import Doc, Token

In [30]:
model_name = "pt_core_news_lg"

try:
    spacy.load(model_name)
    print(f"{model_name} is already installed.")
except OSError:
    print(f"{model_name} is not installed. Downloading...")
    spacy.cli.download(model_name)
    print(f"{model_name} has been downloaded.")

pt_core_news_lg is already installed.


In [31]:
# Download NLTK resources
nltk.download('machado')
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')

# Get Machado's file ids
machado.fileids()

# File id from "Memórias Póstumas de Brás Cubas"
file_id = 'romance/marm05.txt'

[nltk_data] Downloading package machado to
[nltk_data]     /Users/cesarnorena/nltk_data...
[nltk_data]   Package machado is already up-to-date!


# Machado's book

In [32]:
machado_book = Text(machado.words(file_id), name='Memórias Póstumas de Brás Cubas (1881)')

print('# of tokens:', len(machado_book.tokens))
print('# of unique tokens:', len(set(machado_book.tokens)))

# of tokens: 77098
# of unique tokens: 10848


In [33]:
# Experimenting text generation with Machado's book
machado_book.generate(10, text_seed="O personagem mais próximo do Brás Cubas é");

Building ngram index...


O   p e r s o n a g e m   m a i s   p r ó x i m o   d o   B r á s   C u b a s   é que , de dia e de uma bruxa de Shakespeare


In [34]:
import sys


def print_concordance(_book: Text, _characters: List[str], _lines: int = 5):
    # Concordance: an alphabetical list of the words (especially the important ones) present in a text, 
    # usually with citations of the passages in which they are found.

    for character in [char.split() for char in _characters]:
        print('Concordance for:', character)

        concordance_list = _book.concordance_list(character, lines=sys.maxsize, width=100)
        number_of_matches = len(concordance_list)

        min_lines = min(_lines, number_of_matches)
        print(f"Displaying {min_lines} of {number_of_matches} matches:\n")

        for c in concordance_list[:_lines]:
            print(c.line.replace('\n', ''))

        print('\n')

In [35]:
# Main characters to analize from the text
characters = [
    'BRÁS CUBAS',
    'VIRGÍLIA',
    'CONSELHEIRO DUTRA',
    'SABINA',
    'MARCELA',
    'EUGÊNIA',
    'NHÃ-LOLÓ',
    'LOBO NEVES',
    'QUINCAS BORBA',
    'DONA PLÁCIDA',
    'PRUDÊNCIO',
    'COTRIM'
]
print('# of characters:', len(characters), '\n')

print_concordance(machado_book, characters)

# of characters: 12 

Concordance for: ['BRÁS', 'CUBAS']
Displaying 5 of 35 matches:

Romance , Memórias Póstumas de Brás Cubas , 1880 Memórias Póstumas de Brás Cubas Texto
as de Brás Cubas , 1880 Memórias Póstumas de Brás Cubas Texto - fonte : Obra Completa , Machado de A
 primeira edição destas Memórias Póstumas de Brás Cubas foi feita aos pedaços na Revista Brasileira 
vro , perguntava :  As Memórias Póstumas de Brás Cubas são um romance ? Macedo Soares , em carta q
a terra . Ao primeiro respondia já o defunto Brás Cubas ( como o leitor viu e verá no prólogo dele q


Concordance for: ['VIRGÍLIA']
Displaying 5 of 202 matches:

humanas . O que por agora importa saber é que Virgília  chamava - se Virgília  entrou na alcova , 
a importa saber é que Virgília  chamava - se Virgília  entrou na alcova , firme , com a gravidade 
 mais interessante para um moribundo . Saiu ; Virgília deixou - se estar de pé ; durante algum tempo
o sei se em igual dose , mas enfim saciados . Virgília tin

### After analizing the concordance of the names, I decided to use the following list of characters

In [36]:
characters = [
    'Brás Cubas',
    'Virgília',
    'Conselheiro Dutra',
    'Sabina',
    'Marcela',
    'Eugênia',
    'Nhã-loló',
    'Lobo Neves',
    'Quincas Borba',
    'D. Plácida',
    'Prudêncio',
    'Cotrim'
]

print_concordance(machado_book, characters)

Concordance for: ['Brás', 'Cubas']
Displaying 5 of 35 matches:

Romance , Memórias Póstumas de Brás Cubas , 1880 Memórias Póstumas de Brás Cubas Texto
as de Brás Cubas , 1880 Memórias Póstumas de Brás Cubas Texto - fonte : Obra Completa , Machado de A
 primeira edição destas Memórias Póstumas de Brás Cubas foi feita aos pedaços na Revista Brasileira 
vro , perguntava :  As Memórias Póstumas de Brás Cubas são um romance ? Macedo Soares , em carta q
a terra . Ao primeiro respondia já o defunto Brás Cubas ( como o leitor viu e verá no prólogo dele q


Concordance for: ['Virgília']
Displaying 5 of 202 matches:

humanas . O que por agora importa saber é que Virgília  chamava - se Virgília  entrou na alcova , 
a importa saber é que Virgília  chamava - se Virgília  entrou na alcova , firme , com a gravidade 
 mais interessante para um moribundo . Saiu ; Virgília deixou - se estar de pé ; durante algum tempo
o sei se em igual dose , mas enfim saciados . Virgília tinha agora a beleza da v

# SpaCy NLP model

In [37]:
# Load the spaCy Portuguese model (medium size)
language_model = spacy.load('pt_core_news_lg')
language_model.pipe_names

['tok2vec', 'morphologizer', 'parser', 'lemmatizer', 'attribute_ruler', 'ner']

In [38]:
tokenizer = language_model.tokenizer
tokenizer.add_special_case('Brás Cubas', [{'ORTH': 'Brás Cubas'}])
tokenizer.add_special_case('BRÁS CUBAS', [{'ORTH': 'BRÁS CUBAS'}])
tokenizer.add_special_case('Damião Cubas', [{'ORTH': 'Damião Cubas'}])
tokenizer.add_special_case('Luís Cubas', [{'ORTH': 'Luís Cubas'}])
tokenizer.add_special_case('Nhã-loló', [{'ORTH': 'Nhã-loló'}])
tokenizer.add_special_case('Conselheiro Dutra', [{'ORTH': 'Conselheiro Dutra'}])
tokenizer.add_special_case('Luís Dutra', [{'ORTH': 'Luís Dutra'}])
tokenizer.add_special_case('Lobo Neves', [{'ORTH': 'Lobo Neves'}])
tokenizer.add_special_case('Quincas Borba', [{'ORTH': 'Quincas Borba'}])
tokenizer.add_special_case('D. Plácida', [{'ORTH': 'D. Plácida'}])
tokenizer.add_special_case('Dona Plácida', [{'ORTH': 'Dona Plácida'}])

test_doc = language_model(
    "Brás Cubas, BRÁS CUBAS, Nhã-loló, D. Plácida, Dona Plácida, Conselheiro Dutra, Quincas Borba e Lobo Neves.")

print([token.text for token in test_doc])

['Brás Cubas', ',', 'BRÁS CUBAS', ',', 'Nhã-loló', ',', 'D. Plácida', ',', 'Dona Plácida', ',', 'Conselheiro Dutra', ',', 'Quincas Borba', 'e', 'Lobo Neves', '.']


In [39]:
# Processing the raw document with SpaCy portuguese language model
document = language_model(machado.raw(file_id))
tokens = [token for token in document]

# Previous data from NLTK tokens
# of tokens: 77098
# of unique tokens: 10848

print('# of tokens:', len(tokens))
print('# of unique tokens:', len(set([token.text for token in tokens])))

# of tokens: 79214
# of unique tokens: 11536


# Relationships

In [40]:
def find_relationships(_chunks: List[str], _characters: List[str]) -> List[List[str]]:
    relationships = []

    for chunk in _chunks:
        chunk_relationships = []

        for character in _characters:
            if character in chunk:
                chunk_relationships.append(character)

        if len(chunk_relationships) > 0:
            relationships.append(chunk_relationships)

    return relationships

In [41]:
def create_graph(_characters: List[str], _related_characters: List[List[str]]) -> nx.Graph:
    _graph = nx.Graph()
    _graph.add_nodes_from(_characters)

    for sublist in _related_characters:
        for pair in combinations(sublist, 2):
            # If the edge already exists, increment the weight
            if _graph.has_edge(*pair):
                _graph[pair[0]][pair[1]]['weight'] += 1

            # If the edge doesn't exist, create it with weight=1
            else:
                _graph.add_edge(*pair, weight=1)

    return _graph


def show_graph(_graph: nx.Graph, version: str = 'v1'):
    net = Network(notebook=True, width='100%', height='800')
    net.from_nx(_graph)
    net.show(f"relations_bras_cubas_{version}.html")


def print_graph_edges(_graph: nx.Graph):
    for (u, v, wt) in _graph.edges.data('weight'):
        print(f"({u}, {v}, {wt})")
    print()

In [42]:
def betweenness_dataframe(_graph: nx.Graph) -> pd.DataFrame:
    betweenness = nx.betweenness_centrality(_graph, weight='weight')
    dataframe = pd.DataFrame(betweenness.items(), columns=['Character', 'Betweenness'])
    return dataframe.sort_values(by='Betweenness', ascending=False)

In [43]:
def remove_stopwords(_tokens: List[Token]) -> List[Token]:
    return [token for token in _tokens if not token.is_stop]

## v1. Creating the relationship by chunks of 20 tokens

In [44]:
def average(ls: List) -> int:
    return sum(ls) / len(ls)


print('Average size by sentece:', average([len(sentence) for sentence in document.sents]))

Average size by sentece: 18.35782155272306


In [45]:
tokens_without_stopwords = [token.text for token in remove_stopwords(tokens)]

chunk_size = 20
words_chunks = [tokens_without_stopwords[i:i + chunk_size] \
                for i in range(0, len(tokens_without_stopwords), chunk_size)]

related_characters = find_relationships(words_chunks, characters)

print('# of relationships found:', len(related_characters))
# related_characters

# of relationships found: 466


In [46]:
graph = create_graph(characters, related_characters)

weights = graph.edges.data('weight')
pd.DataFrame(weights, columns=['Source', 'Target', 'Weight']).sort_values(by='Weight', ascending=False)

,Source,Target,Weight
9,Virgília,D. Plácida,10
6,Virgília,Lobo Neves,9
12,Sabina,Cotrim,8
8,Virgília,Quincas Borba,5
7,Virgília,Sabina,3
15,Sabina,Nhã-loló,2
5,Virgília,Marcela,2
1,Brás Cubas,Virgília,2
17,Lobo Neves,Quincas Borba,1
16,Lobo Neves,D. Plácida,1


In [47]:
show_graph(graph, 'v1');

relations_bras_cubas_v1.html


In [48]:
betweenness_dataframe(graph)

,Character,Betweenness
1,Virgília,0.463636
0,Brás Cubas,0.172727
3,Sabina,0.054545
8,Quincas Borba,0.036364
4,Marcela,0.018182
7,Lobo Neves,0.018182
2,Conselheiro Dutra,0.000000
5,Eugênia,0.000000
6,Nhã-loló,0.000000
9,D. Plácida,0.000000


## v2. Creating the relationship by sentences

In [49]:
sentence_chunks = [span.text for span in document.sents]

related_characters = find_relationships(sentence_chunks, characters)

print('# of relationships found:', len(related_characters))
# related_characters

# of relationships found: 498


In [50]:
graph = create_graph(characters, related_characters)

weights = graph.edges.data('weight')
pd.DataFrame(weights, columns=['Source', 'Target', 'Weight']).sort_values(by='Weight', ascending=False)

,Source,Target,Weight
5,Virgília,Lobo Neves,8
8,Virgília,D. Plácida,6
11,Sabina,Cotrim,4
4,Virgília,Marcela,3
7,Virgília,Quincas Borba,3
13,Sabina,Nhã-loló,2
2,Brás Cubas,Marcela,2
16,Lobo Neves,D. Plácida,2
1,Brás Cubas,Virgília,2
0,Brás Cubas,Eugênia,1


In [51]:
show_graph(graph, version='v2');

relations_bras_cubas_v2.html


In [52]:
betweenness_dataframe(graph)

,Character,Betweenness
1,Virgília,0.284848
0,Brás Cubas,0.151515
3,Sabina,0.048485
4,Marcela,0.036364
7,Lobo Neves,0.015152
8,Quincas Borba,0.015152
11,Cotrim,0.012121
2,Conselheiro Dutra,0.000000
5,Eugênia,0.000000
6,Nhã-loló,0.000000


# Characters style similarity

In [53]:
def find_sentences(_model: Language, _document: Doc, _word: str) -> Doc:
    matcher = Matcher(_model.vocab)
    matcher.add(_word, [[{"LOWER": _word.lower()}]])
    matches = matcher(_document)

    sentences = [_document[start:end].sent.text for _, start, end in matches]

    return _model(' '.join(sentences))


bras_cubas = find_sentences(language_model, document, 'Brás Cubas')
characters_similarity = {}

for character in characters[1:]:
    character_doc = find_sentences(language_model, document, character)
    characters_similarity[character] = bras_cubas.similarity(character_doc)

pd.DataFrame(characters_similarity.items(), columns=['Character', 'Similarity']) \
    .sort_values(by='Similarity', ascending=False)

,Character,Similarity
6,Lobo Neves,0.980151
7,Quincas Borba,0.975325
3,Marcela,0.973849
0,Virgília,0.973290
8,D. Plácida,0.970998
10,Cotrim,0.970131
5,Nhã-loló,0.965064
4,Eugênia,0.956754
2,Sabina,0.954632
9,Prudêncio,0.930684


### Failed attempt of training the tok2vec model to create a vector from a single character's name 

In [54]:
from spacy.util import minibatch, compounding
from spacy.training import Example
import random

new_nlp = spacy.blank("pt")
new_nlp.add_pipe("tok2vec")

texts = [span.text for span in document.sents]
annotations = [{"entities": []} for t in texts]

# Initialize the model
optimizer = new_nlp.initialize()

examples = [Example.from_dict(new_nlp(text), annot) for text, annot in zip(texts, annotations)]

# Number of epochs
epochs = 10

for i in range(epochs):
    # Shuffle the examples at the beginning of each epoch
    random.shuffle(examples)
    losses = {}

    # Create batches using spaCy's minibatch utility function
    batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))

    for batch in batches:
        # Update the model
        new_nlp.update(batch, sgd=optimizer, losses=losses)

    print(f"Losses at iteration {i}: {losses}")


Losses at iteration 0: {'tok2vec': 0.0}
Losses at iteration 1: {'tok2vec': 0.0}
Losses at iteration 2: {'tok2vec': 0.0}
Losses at iteration 3: {'tok2vec': 0.0}
Losses at iteration 4: {'tok2vec': 0.0}
Losses at iteration 5: {'tok2vec': 0.0}
Losses at iteration 6: {'tok2vec': 0.0}
Losses at iteration 7: {'tok2vec': 0.0}
Losses at iteration 8: {'tok2vec': 0.0}
Losses at iteration 9: {'tok2vec': 0.0}
